# Image Processing Lab11

## Setup

In [ ]:
!apt update && apt install -y ffmpeg libsm6 libxext6
!pip install tensorflow
!pip install opencv-python

Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://security.debian.org/debian-security buster/updates InRelease [65.4 kB]
Get:3 http://deb.debian.org/debian buster-updates InRelease [51.9 kB]
Get:4 http://deb.debian.org/debian buster/main amd64 Packages [7,911 kB]
Get:5 http://deb.debian.org/debian buster-updates/main amd64 Packages [8,796 B]
Get:6 http://security.debian.org/debian-security buster/updates/main amd64 Packages [317 kB]
Fetched 8,476 kB in 1s (5,724 kB/s)



29 packages can be upgraded. Run 'apt list --upgradable' to see them.



libsm6 is already the newest version (2:1.2.3-1).
libsm6 set to manually installed.
libxext6 is already the newest version (2:1.3.3-1+b2).
libxext6 set to manually installed.
The following additional packages will be installed:
  i965-va-driver intel-media-va-driver libaacs0 libaom0 libasound2
  libasound2-data libass9 libasyncns0 libavc1394-0 libavcodec58 libavdevice58
  libavfilter7 libavformat58 libavresample4 libavutil5

## Import

In [ ]:
import glob
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
import keras
from keras.preprocessing import image
from keras.layers import Dense,GlobalAveragePooling2D
from tensorflow.keras.applications.mobilenet import MobileNet
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

## 11.1 Prepare Image Data(Real, Fake)

### 11.1.1 Prepare Real Image

In [ ]:
FILE_PATTERN = "/datasets/dataset/art-image-dataset/**/*/*"

#List all filename in art image path
img_size = (32, 32, 3)
filenames= glob.glob(FILE_PATTERN)

train_dataset = [] # read image from dataset and resize to train_dataset.shape = img_size using interpolation="bilinear"
for file in filenames:
    print(file)
    img = image.load_img(file, target_size=img_size, interpolation="bilinear")
    train_dataset.append(img)
#rescale train_dataset values [0, 255] -> [-1, 1]

print(train_dataset)

/datasets/dataset/art-image-dataset/Abstract_gallery/Abstract_gallery/Abstract_image_148.jpg
/datasets/dataset/art-image-dataset/Abstract_gallery/Abstract_gallery/Abstract_image_1498.jpg
/datasets/dataset/art-image-dataset/Abstract_gallery/Abstract_gallery/Abstract_image_1455.jpg
/datasets/dataset/art-image-dataset/Abstract_gallery/Abstract_gallery/Abstract_image_1509.jpg
/datasets/dataset/art-image-dataset/Abstract_gallery/Abstract_gallery/Abstract_image_1353.jpg
/datasets/dataset/art-image-dataset/Abstract_gallery/Abstract_gallery/Abstract_image_1364.jpg
/datasets/dataset/art-image-dataset/Abstract_gallery/Abstract_gallery/Abstract_image_1321.jpg
/datasets/dataset/art-image-dataset/Abstract_gallery/Abstract_gallery/Abstract_image_1396.jpg
/datasets/dataset/art-image-dataset/Abstract_gallery/Abstract_gallery/Abstract_image_1368.jpg
/datasets/dataset/art-image-dataset/Abstract_gallery/Abstract_gallery/Abstract_image_1335.jpg
/datasets/dataset/art-image-dataset/Abstract_gallery/Abstract

In [ ]:
# Random Selected images
def generate_real_samples(dataset, n_samples):
    ix = randint(0, dataset.shape[0], n_samples) # random image index
    # retrieve selected images
    X = dataset[ix] # get image from random pick ix
    # generate y = 1 for 'real' class labels
    y #-> (y.shape = (n_samples,1))
return X, y

In [ ]:
#Display Random Real Images
X_real, y_real = generate_real_samples(train_dataset, n_samples)
cv.imshow(X_real) # convert dtype to uint8

## 11.1.2 Prepare Fake Image initial for training discriminator(D)

In [ ]:
def generate_fake_samples(n_samples, target_size=(32,32,3)):
    h,w,c = target_size
    # generate 1D uniform random numbers in [0,1]
    X = rand() #-> X.shape = (n_samples * h * w * 3)
    # rescale X value from [0, 1] to [-1, 1]
    X # Xmax = 1, Xmin = -1
    # reshape X from 1D to 4D
    X # X.shape = (n_samples, h, w, c)
    # generate y = 0 for 'fake' class label
    y # y.shape = (n_samples,1)
    return X, y

In [ ]:
# generate fake image samples
X_fake, y_fake = generate_fake_samples(n_samples, target_size=img_size)
# Display Random Real Images
cv.imshow(X_fake) # rescale to [0, 255] -> dtype = uint8

## 11.2 Prepare Discriminator, Generator Model

### 11.2.1 Create Discriminator Model (D) and Training

In [ ]:
# Discriminator Model
def define_discriminator(in_shape=(32,32,3)):
    model = Sequential()
    # normal
    model.add(Conv2D(CNN_L1, (3,3), padding='same', input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    # downsample
    model.add(Conv2D(CNN_L2, (3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # downsample
    model.add(Conv2D(CNN_L3, (3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # downsample
    model.add(Conv2D(CNN_L4, (3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # classifier
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    model.compile(loss= Train_loss, optimizer=opt, metrics=['accuracy'])
    return model

In [ ]:
# train the discriminator model
def train_discriminator(model, dataset, n_iter=60, n_batch=128, target_size=(32,32,3)):
    half_batch = int(n_batch / 2)
    # manually enumerate epochs
    for i in range(n_iter):
    # get randomly selected 'real' samples
    X_real, y_real = generate_real_samples(dataset, half_batch)
    # update discriminator on real samples
    _, real_acc = model.train_on_batch(X_real, y_real)
    # generate 'fake' examples
    X_fake, y_fake = generate_fake_samples(half_batch, target_size=img_size)
    # update discriminator on fake samples
    _, fake_acc = model.train_on_batch(X_fake, y_fake)
    # summarize performance
    print(real_acc, fake_acc)

### 11.2.1 Create Model and pre-train Discriminator (D)

In [ ]:
# define model
d_model = define_discriminator(img_size)

# summarize the model
d_model.summary()

# Training Discriminator
train_discriminator(d_model, dataset, target_size = img_size)

### 11.2.2 Create Generator Model (G)

In [ ]:
# define the standalone generator model
def define_generator(latent_dim, discr_out = (256,4,4)):
    model = Sequential()
    # foundation for 4x4 image
    c, w, h = discr_out
    n_nodes = c * w * h
    model.add(Dense(n_nodes, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((w, h, c)))
    # upsample to 8x8
    model.add(Conv2DTranspose(128, (w,h), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 16x16
    model.add(Conv2DTranspose(128, (w,h), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 32x32
    model.add(Conv2DTranspose(128, (w,h), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # output layer
    model.add(Conv2D(3, (3,3), activation='tanh', padding='same'))
    return model

### 11.2.2 Create Model and pre-train Discriminator(D)

In [ ]:
# define the generator model
g_model = define_generator(latent_dim, discr_out = (256,mini_size[0],mini_size[1]))

# summarize the model
g_model.summary()

## 11.3 GAN Training and Prediction

### 11.3.1 Create GAN Model (G, D) and Training

In [ ]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
    # make weights in the discriminator not trainable
    d_model.trainable = False
    # connect them
    model = Sequential()
    # add generator
    model.add(g_model)
    # add the discriminator
    model.add(d_model)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

In [ ]:
# update generate fake image using generator prediction
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(g_model, latent_dim, n_samples):
    # generate points in latent space
    x_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    X = g_model.predict(x_input)
    # create 'fake' class labels (0)
    y # y = 0 with shape (n_sample, )
    return X, y

In [ ]:
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=EP, n_batch=128):
    bat_per_epo = int(dataset.shape[0] / n_batch)
    half_batch = int(n_batch / 2)
    # manually enumerate epochs
    for i in range(n_epochs):
        # enumerate batches over the training set
        for j in range(bat_per_epo):
            # get randomly selected 'real' samples
            X_real, y_real = generate_real_samples(dataset, half_batch)
            d_loss1, _ = d_model.train_on_batch(X_real, y_real) # update discriminator model weights
            # generate 'fake' examples
            X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            d_loss2, _ = d_model.train_on_batch(X_fake, y_fake) # update discriminator model weights
            # prepare points in latent space as input for the generator
            X_gan = generate_latent_points(latent_dim, n_batch)
            y_gan # # create inverted labels for the fake samples y = 0 with shape (n_batch, )
            # update the generator via the discriminator's error
            g_loss = gan_model.train_on_batch(X_gan, y_gan)
            print(d_loss1, d_loss2, g_loss) # summarize loss on this batch
            # evaluate the model performance, every Nepoch
            if (i+1) % Nepoch == 0:
                summarize_performance(i, g_model, d_model, dataset, latent_dim)

In [ ]:
# evaluate the discriminator, plot generated images, save generator model
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=150):
# prepare real samples
    X_real, y_real = generate_real_samples(dataset, n_samples)
    # evaluate discriminator on real examples
    _, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
    # prepare fake examples
    x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
    # evaluate discriminator on fake examples
    _, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
    # summarize discriminator performance
    print(acc_real , acc_fake)
    # save plot
    save_plot(x_fake, epoch)
    # save the generator model tile file
    filename = 'generator_model_%03d.h5' % (epoch+1)
    g_model.save(filename)

In [ ]:
# create the gan
gan_model = define_gan(g_model, d_model)
# summarize gan model
gan_model.summary()

In [ ]:
# train model
train(g_model, d_model, gan_model, dataset, latent_dim)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=9f674496-f7ae-4072-876a-2ff328de42ce' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>